# Imports and Setup

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense, InputLayer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import LeakyReLU

USE_GPU = True

2024-03-07 14:23:16.235184: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-07 14:23:16.258426: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-07 14:23:16.258450: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-07 14:23:16.259035: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-07 14:23:16.263811: I tensorflow/core/platform/cpu_feature_guar

In [2]:
if USE_GPU:
    #tf.debugging.set_log_device_placement(True)
    sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

2024-03-07 14:23:18.975986: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 14:23:18.997721: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 14:23:18.997952: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 3050 Ti Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6



2024-03-07 14:23:19.186115: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 14:23:19.186191: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 14:23:19.186197: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2022] Could not identify NUMA node of platform GPU id 0, defaulting to 0.  Your kernel may not have been built with NUMA support.
2024-03-07 14:23:19.186221: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 14:23:19.186235: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created

In [3]:
## Define path to data and parameters for loader
TRAIN_PATH = './data/training/'
TEST_PATH = './data/testing/'
LABELS_PATH = './data/training_labels.csv'

TEST_SIZE = 0.25
BATCH_SIZE = 32
#IMG_HEIGHT = 180
#IMG_WIDTH = 180

N_CLASSES = 5

## Encoding labels
LABEL_ENCODING = {
    'daisy': 0,
    'dandelion': 1,
    'rose': 2,
    'sunflower': 3,
    'tulip': 4
}

Read in data and assign labels

In [4]:
## Get labels for train and test data
labels_df = pd.read_csv(LABELS_PATH)
labels_df['full_id'] = [os.path.join('/home/scottbrown/byu/stat486/projects/flower-image-classification/data/training/training', l) for l in labels_df.ID]

In [5]:
# labels_df['target_encoded'] = labels_df['target'].copy() #.map(LABEL_ENCODING)

# train_labels = list(zip(labels_df['ID'], labels_df['target_encoded']))
# test_labels = list(zip(labels_df['ID'], labels_df['target_encoded']))

# ## Sort (in place) according to the alphanumeric order of the image file paths.
# train_labels.sort(key=lambda x: x[0])
# test_labels.sort(key=lambda x: x[0])

## Change first item in each tuple to index
# # train_labels = [(i, l[1]) for i, l in enumerate(train_labels)]
# # test_labels = [(i, l[1]) for i, l in enumerate(test_labels)]

In [6]:
## NOTE: `flow_from_dataframe` via ImageDataGenerator is deprecated
# In the future, organize directory structure for `flow_from_directory`
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=TEST_SIZE)
train_gen = datagen.flow_from_dataframe(
    labels_df, 
    directory=TRAIN_PATH, 
    subset='training',
    x_col='full_id', 
    y_col='target', 
    class_mode='categorical', 
    batch_size=BATCH_SIZE)

test_gen = datagen.flow_from_dataframe(
    labels_df, 
    directory=TRAIN_PATH, 
    subset='validation',
    x_col='full_id', 
    y_col='target', 
    class_mode='categorical', 
    batch_size=BATCH_SIZE)

Found 2591 validated image filenames belonging to 5 classes.
Found 863 validated image filenames belonging to 5 classes.


In [7]:
# train = tf.keras.utils.image_dataset_from_directory(
#   TRAIN_PATH,
#   labels=train_labels,
#   #label_mode = 'categorical',
#   validation_split=TEST_SIZE,
#   subset='training',
#   seed=9, # Joe Burrow
#   #image_size=(img_height, img_width),
#   batch_size=BATCH_SIZE)

# test = tf.keras.utils.image_dataset_from_directory(
#   TRAIN_PATH,
#   labels=test_labels,
#   #label_mode = 'categorical',
#   validation_split=TEST_SIZE,
#   subset='validation',
#   seed=9, # Joe Burrow
#   #image_size=(img_height, img_width),
#   batch_size=BATCH_SIZE)

# Preview images

In [8]:
# ## Recreate the above code for train_gen generator object
# class_names = list(train_gen.class_indices.keys())
# for images, labels in train_gen:
#   plt.figure(figsize=(10, 10))
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     lab = class_names[labels[i].argmax()]
#     plt.imshow(images[i])
#     plt.title(lab)
#     plt.axis("off")
#   break

# Create model

In [9]:
cnn = tf.keras.Sequential([
    ## Preprocessing - now handled in data loading
    # tf.keras.layers.Rescaling(1./255), # Normalize RGB values
    ## 3 Convolutional => MaxPooling layers
    tf.keras.layers.Conv2D(48, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.4),

    tf.keras.layers.Conv2D(48, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.4),

    tf.keras.layers.Conv2D(48, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.4),

    ## 1 Fully connected layer
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),#LeakyReLU(alpha=0.01)),
    
    ## Output layer
    tf.keras.layers.Dense(N_CLASSES, activation='softmax')
])

2024-03-07 14:23:19.293648: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 14:23:19.293834: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 14:23:19.293910: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 14:23:19.294079: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 14:23:19.294099: I external/local_xla/xla/stream_executor

In [10]:
cnn.compile(
  optimizer='adam',
  loss=tf.keras.losses.CategoricalCrossentropy(),
  metrics=['accuracy'])

# Fit model

In [11]:
cnn.fit(
  train_gen,
  validation_data=test_gen,
  epochs=20
)

2024-03-07 14:23:19.547386: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-07 14:23:19.676780: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-07 14:23:20.847070: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch 1/20


2024-03-07 14:23:22.256057: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-03-07 14:23:23.458611: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f45e49fe800 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-07 14:23:23.458641: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Ti Laptop GPU, Compute Capability 8.6
2024-03-07 14:23:23.463041: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1709846603.527724   15538 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2024-03-07 14:23:34.291671: W ext

27/81 [=========>....................] - ETA: 6s - loss: 2.3517 - accuracy: 0.2199

2024-03-07 14:23:50.977071: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.53GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


28/81 [=========>....................] - ETA: 32s - loss: 2.3252 - accuracy: 0.2201

2024-03-07 14:23:51.943535: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.53GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


81/81 [==============================] - 37s 298ms/step - loss: 1.6840 - accuracy: 0.3342 - val_loss: 1.2941 - val_accuracy: 0.4426
Epoch 2/20
81/81 [==============================] - 10s 128ms/step - loss: 1.1701 - accuracy: 0.5048 - val_loss: 1.1719 - val_accuracy: 0.5597
Epoch 3/20
81/81 [==============================] - 10s 127ms/step - loss: 1.0809 - accuracy: 0.5646 - val_loss: 1.1093 - val_accuracy: 0.5701
Epoch 4/20
81/81 [==============================] - 10s 126ms/step - loss: 1.0024 - accuracy: 0.6040 - val_loss: 1.0719 - val_accuracy: 0.5771
Epoch 5/20
81/81 [==============================] - 10s 126ms/step - loss: 0.9283 - accuracy: 0.6357 - val_loss: 1.0031 - val_accuracy: 0.5794
Epoch 6/20
81/81 [==============================] - 10s 126ms/step - loss: 0.8321 - accuracy: 0.6820 - val_loss: 1.0321 - val_accuracy: 0.5817
Epoch 7/20
81/81 [==============================] - 10s 126ms/step - loss: 0.7299 - accuracy: 0.7171 - val_loss: 0.9597 - val_accuracy: 0.5991
Epoch 8/20

In [12]:
test_eval = cnn.evaluate(test_gen)#X_test, y_test, verbose=0)

print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

27/27 [==============================] - 1s 42ms/step - loss: 2.0138 - accuracy: 0.5979
Test loss: 2.0138320922851562
Test accuracy: 0.5979142785072327
